In [1]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
cd drive/MyDrive/Deep_Learning_Study/GAN/DCGAN

/content/drive/MyDrive/Deep_Learning_Study/GAN/DCGAN


In [16]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
latent_size = 64
hidden_size = 256
img_size = 64
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

In [5]:
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [6]:
transform = transforms.Compose([
                transforms.Resize((img_size,img_size)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5],   # 1 for greyscale channels
                                     std=[1.0])])

In [7]:
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [8]:
mnist = torchvision.datasets.MNIST(root='./MNIST_DATASET',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [19]:
class generator(nn.Module):

    def __init__(self, d=128):
        super(generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(64, d*8, 4, 1, 0),
            nn.BatchNorm2d(d*8),
            nn.ReLU(True),
            nn.ConvTranspose2d(d*8, d*4, 4, 2, 1),
            nn.BatchNorm2d(d*4),
            nn.ReLU(True),
            nn.ConvTranspose2d(d*4, d*2, 4, 2, 1),
            nn.BatchNorm2d(d*2),
            nn.ReLU(True),
            nn.ConvTranspose2d(d*2, d, 4, 2, 1),
            nn.BatchNorm2d(d),
            nn.ReLU(True),
            nn.ConvTranspose2d(d, 1, 4, 2, 1),
            nn.Tanh()
            )

    # forward method
    def forward(self, input):
        return self.main(input)

In [20]:
class discriminator(nn.Module):
    # initializers
    def __init__(self, d = 128):
        super(discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(1, d, 4, 2, 1),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(d, d*2, 4, 2, 1),
            nn.BatchNorm2d(d*2),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(d*2, d*4, 4, 2, 1),
            nn.BatchNorm2d(d*4),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(d*4, d*8, 4, 2, 1),
            nn.BatchNorm2d(d*8),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(d*8, 1, 4, 1, 0),
            nn.Sigmoid()
        )

    # forward method
    def forward(self, input):
        return self.main(input)

In [21]:
G = generator(64).to(device)
D = discriminator(64).to(device)

G.apply(weights_init)
D.apply(weights_init)

criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [22]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

In [23]:
def weights_init(m):
    classname = m.__class__.__name__ 
    if classname.find('Conv') != -1: 
        nn.init.normal_(m.weight.data, 0.0,0.02)
    elif classname.find('BatchNorm') != -1: 
        nn.init.normal_(m.weight.data,1.0,0.02) 
        nn.init.constant_(m.bias.data,0)


In [ ]:
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        D.zero_grad()
        images = images.to(device)

        real_labels = torch.ones(batch_size).to(device)
        fake_labels = torch.zeros(batch_size).to(device)

        outputs = D(images).squeeze()
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs

        z = torch.randn((batch_size, latent_size,1,1)).to(device)

        fake_images = G(z)
        outputs = D(fake_images).squeeze()
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs

        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        d_optimizer.step()

        G.zero_grad()

        z = torch.randn((batch_size, latent_size,1,1)).to(device)
        fake_images = G(z)
        outputs = D(fake_images).squeeze()
        
        g_loss = criterion(outputs, real_labels)
         
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
        if (epoch+1) == 1:
            save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))

        save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

Epoch [0/200], Step [200/600], d_loss: 0.0095, g_loss: 9.0432, D(x): 1.00, D(G(z)): 0.01
Epoch [0/200], Step [400/600], d_loss: 0.0040, g_loss: 7.3848, D(x): 1.00, D(G(z)): 0.00
Epoch [0/200], Step [600/600], d_loss: 0.1519, g_loss: 7.8544, D(x): 0.92, D(G(z)): 0.00
Epoch [1/200], Step [200/600], d_loss: 0.0218, g_loss: 7.6414, D(x): 0.98, D(G(z)): 0.01
Epoch [1/200], Step [400/600], d_loss: 0.0300, g_loss: 7.1421, D(x): 0.98, D(G(z)): 0.00
Epoch [1/200], Step [600/600], d_loss: 0.0437, g_loss: 6.9935, D(x): 1.00, D(G(z)): 0.04
Epoch [2/200], Step [200/600], d_loss: 0.0177, g_loss: 7.5285, D(x): 0.99, D(G(z)): 0.00
Epoch [2/200], Step [400/600], d_loss: 0.0164, g_loss: 7.0900, D(x): 0.99, D(G(z)): 0.01
Epoch [2/200], Step [600/600], d_loss: 0.0517, g_loss: 5.4173, D(x): 0.98, D(G(z)): 0.03
Epoch [3/200], Step [200/600], d_loss: 0.1986, g_loss: 7.0964, D(x): 0.99, D(G(z)): 0.15
Epoch [3/200], Step [400/600], d_loss: 0.0490, g_loss: 5.9579, D(x): 0.96, D(G(z)): 0.01
Epoch [3/200], Step [

In [ ]:
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')